## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
data.info()
print(data.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124

### Вывод

Всего в датасете 21525 записей. 

Неполные данные встречаются в столбцах а) об общем трудовом стаже; б) о ежемесячном доходе. Можно предположить, что не заполнены эти поля у одних и тех же людей, т.к. данные относятся к одной сфере, а количество пропусков в обоих столбцах идентично (по 2174), но эту гипотезу мы проверим в дальнейшем. 

Наибольшее беспокойство вызывает поле общего трудового стажа - данные явно требуют серьезной работы (встречаются отрицательные, дробные и нереалистичные значения).

Остальные поля требуют незначительного вмешательства.

Также обращает на себя внимание, что пропуски и некачественные данные есть только в столбцах с типом float - возможно, существуют технические ошибки при выгрузке данных этого типа или при работе с ним на стороне банка.

### Шаг 2. Предобработка данных

### Обработка пропусков

Сначала проверим, насколько пересекаются пропуски в данных об общем трудовом стаже и доходе.

In [2]:
data_grouped=data[(data['days_employed'].isnull())&(data['total_income'].isnull())]
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Как мы и предполагали, пропуски в обоих столбцах всегда идут рука об руку.

Теперь попробуем установить корреляцию между наличием пропусков и значениями других полей

In [3]:
#С какими факторами может коррелировать отсутствие данных о стаже и доходе? Попробуем установить корреляцию с образованием.
print(data['education_id'].value_counts())
print(data_grouped['education_id'].value_counts())
#Существенной корреляции нет. Попробуем установить корреляцию с типом занятости
print(data['income_type'].value_counts())
print(data_grouped['income_type'].value_counts())
#Тоже нет. Есть ли корреляция с наличием долга?
print(data['debt'].value_counts())
print(data_grouped['debt'].value_counts())
#Также нет.

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
0    19784
1     1741
Name: debt, dtype: int64
0    2004
1     170
Name: debt, dtype: int64


Судя по всему, пропуски носят полностью случайный характер.

Переходим к заполнению пропусков.

In [4]:
#Никаких работ над столбцом с общим трудовым стажем мы пока не проводили, но заполним пропуски медианными значениями - это не должно внести искажений в наши данные
avg_days_employed=data['days_employed'].median()
data['days_employed']=data['days_employed'].fillna(value=avg_days_employed)


In [5]:
#Напишем функцию, вычисляющую медианный доход по каждой из категорий занятости и присваивающую этот медианный доход тем представителям категорий, у которых он не указан
def set_average_income(income_type):
    avg_total_income=data.loc[data.loc[:,'income_type']==income_type]['total_income'].median()
    data.loc[(data['income_type'] == income_type)&(data['total_income'].isna()), 'total_income'] = avg_total_income
#Применим эту функцию к тем типам занятости, которые встречаются среди записей с пропущенными данными 
set_average_income('сотрудник')
set_average_income('компаньон')
set_average_income('пенсионер')
set_average_income('госслужащий')
set_average_income('предприниматель')


### Вывод

Мы выяснили, что пропуски в данных носят случайный характер, и в случае со столбцом трудового стажа - заполнили их медианными значениями. Скорее всего, это неидеальное решение, но поскольку этот столбец никак не поможет нам в ответе на вопросы, которые составляют цель исследования, не будем тратить на него слишком много времени.

В случае с ежемесячным доходом мы вычислили медианный доход по категориям занятости и заполнили отсутствующие значения своей медианой для каждой категории. Поскольку этот столбец понадобится нам в дальнейшем, здесь требуемая точность выше.

Причины появления пропусков - либо в непредоставлении документов о занятости, либо в технических недочетах при обработке этих данных.

### Замена типа данных

Столбец общего дохода имеет вещественный тип данных, что не только избыточно для наших целей, но и неверно: значения имют 6 знаков после запятой, а не два. Учитывая, что речь идет о суммах в несколько сотен тысяч рублей, вполне достаточно точности, обеспечиваемой целочисленным типом данных. Проведем замену.

In [6]:
data['total_income']=data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Из столбцов, которые представляют интерес для решения нашей задачи, проблемный тип данных был в одном - ежемесячном доходе. Мы сменили тип данных с вещественного на целочисленный, для наших задач этого достаточно. Вещественный тип данных в столбце "Общий трудовой стаж в днях" также очевидно неправилен, но учитывая, что привести этот столбец к полностью нормальному виду имеющимися у нас средствами невозможно, а в наших расчетах он не используется, наиболее оптимально ничего с ним не делать. 

### Обработка дубликатов

Пройдемся по столбцам со строковым типом данных в поисках по-разному написанных дубликатов. Начнем со столбца education 

In [7]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Как видим, таких немало. Исправим написание, приведя все значения к нижнему регистру.

In [8]:
data['education']=data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Надо убедиться, что написание в разных регистрах не повлияло на соотнесение значений в столбцах education и education_id. 

В норме среднему образованию должен соответствовать education_id 1. Посмотрим, всегда ли это так.

In [9]:
data_education_middle=data[(data['education']=='среднее')&(data['education_id']!=1)]
print(data_education_middle)

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Неправильно обработанных значений нет. Повторим то же самое для других типов образования.

In [10]:
data_education_higher=data[(data['education']=='высшее')&(data['education_id']!=0)]
print(data_education_higher)
data_education_higher_unfinished=data[(data['education']=='неоконченное высшее')&(data['education_id']!=2)]
print(data_education_higher_unfinished)
data_education_low=data[(data['education']=='начальное')&(data['education_id']!=3)]
print(data_education_low)
data_education_phd=data[(data['education']=='ученая степень')&(data['education_id']!=4)]
print(data_education_phd)

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


С остальными значениями тоже все нормально. Перейдем к графе family_status

In [11]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Здесь дубликатов нет. Переходим к столбцу gender.

In [12]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Здесь дубликатов тоже нет, но есть одно ошибочное значение XNA. Посмотрим, есть ли еще что-то необычное в строке, содержащей это значение.

In [13]:
print(data[data['gender']=='XNA'])

       children  days_employed  dob_years            education  education_id  \
10701         0   -2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

       total_income               purpose  
10701        203905  покупка недвижимости  


Исходя из имеющихся данных и того, что женщин в выборке почти в 2 раза больше, чем мужчин, предположим, что это женщина.

In [14]:
data.loc[data['gender']=='XNA', 'gender'] = 'F'
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Столбец income_type мы уже просматривали выше, там дубликатов нет. В столбце purpose данные не унифицированы, к нему мы перейдем позже. Осталось найти полные дубликаты строк и удалить их.

In [15]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Мы удалили 71 дублирующуюся строку. Поскольку набор данных в строке достаточно подробный и включает в себя такие индивидуальные характеристики, как рабочий стаж, возраст и доход, полностью идентичные строки с очень большой вероятностью действительно являются дублями - вероятность, что эти записи принадлежат разным людям, крайне низка.

### Вывод

Мы избавились от дубликатов, связанных с разным написанием категорий образования, а также от полностью дублирующихся строк. Появление вторых скорее всего связано с повторым внесением в систему одних и тех же заявок. Также убедились, что в других категориях дубликатов нет.

### Лемматизация

Перейдем к столбцу purpose. Он нужен нам для ответа на один из стоящих перед нами вопросов, но данные в нем не унифицированы: ответы заполнялись в свободной форме. Поэтому проведем лемматизацию.

In [16]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lemmas']=data['purpose'].apply(m.lemmatize)

### Вывод

Мы лемматизировали данные в столбце purpose с исвпользованием библиотеки pymystem3 и сохранили полученные леммы в отдельный столбец purpose_lemmas. Это позволит нам работать с данными по целям взятия кредита, начнем это делать мы на следующем шаге.

### Категоризация данных

Из интересующих нас столбцов с данными один - семейное положение - не требует дополнительных операций. Уровень дохода и причина взятия кредита требуют категоризации. Также надо изучить стоблец количества детей на предмет качества данных.


Посмотрим, какие значения есть в столбце количества детей.

In [17]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

У 47 человек есть -1 ребенок, а у 76-и - 20. Учитывая, что ни у одного человека нет ни 19, ни 18 детей, вероятно, перед нами ошибка. Вряд ли мы сможем установить верное число детей для каждой из ошибочных записей, но в данном случае наша задача - отнести эти записи к одной из двух категорий: "есть дети" или "нет детей". Посмотрим на усредненные характеристики каждой из этих групп, чтобы понять, к какой из них ближе записи с неверными значениями. Выберем две ключевые связанные с этим характеристики - возраст и семейное положение. Начнем с тех, у кого нет детей.

In [18]:
data_no_children=data[data['children']==0]
print(data_no_children['dob_years'].median())
print(data_no_children['family_status'].value_counts())

48.0
женат / замужем          7468
гражданский брак         2730
Не женат / не замужем    2262
вдовец / вдова            847
в разводе                 784
Name: family_status, dtype: int64


Теперь посмотрим на тех, у кого дети есть.

In [19]:
data_children=data[data['children']!=0]
print(data_children['dob_years'].median())
print(data_children['family_status'].value_counts())

37.0
женат / замужем          4871
гражданский брак         1421
Не женат / не замужем     548
в разводе                 411
вдовец / вдова            112
Name: family_status, dtype: int64


Как видим, те, у кого есть дети, в среднем моложе и чаще состоят в браке. Теперь посмотрим на тех, у кого -1 ребенок, чтобы понять, на какую из групп они больше похожи.

In [20]:
data_error_1=data[data['children']==-1]
print(data_error_1['dob_years'].median())
print(data_error_1['family_status'].value_counts())

41.0
женат / замужем          29
гражданский брак          5
Не женат / не замужем     5
в разводе                 4
вдовец / вдова            4
Name: family_status, dtype: int64


Значения ближе к тем, у кого есть дети. Теперь посмотрим на тех, у кого 20 детей.

In [21]:
data_error_20=data[data['children']==20]
print(data_error_20['dob_years'].median())
print(data_error_20['family_status'].value_counts())

41.5
женат / замужем          49
гражданский брак         12
Не женат / не замужем     9
вдовец / вдова            4
в разводе                 2
Name: family_status, dtype: int64


Они по своим характеристикам также ближе к тем, у кого есть дети. Будем считать, что обе категории на самом деле имеют детей, их точное количество в данном случае для нас неважно. 

Теперь перейдем к категоризации столбца total_income. Сначала изучим разброс значений.

In [22]:
print('Минимальный доход:',data['total_income'].min())
print('Максимальный доход:',data['total_income'].max())
print('Медианный доход:',data['total_income'].median())

Минимальный доход: 20667
Максимальный доход: 2265604
Медианный доход: 142594.0


Как видим, разброс весьма велик. Попробуем посчитать количество значений вокруг нашей медины. Сколько людей имеет доход от 100 до 200 тыс. рублей в месяц?

In [23]:
data_average=data[(data['total_income']>=100000)&(data['total_income']<=200000)]['total_income']
data_min=data[(data['total_income']<100000)]['total_income']
data_max=data[(data['total_income']>200000)]['total_income']
print('Людей с доходом менее 100 тыс. рублей:',data_min.count())
print('Людей с доходом от 100 до 200 тыс. рублей:',data_average.count())
print('Людей с доходом более 200 тыс. рублей:',data_max.count())

Людей с доходом менее 100 тыс. рублей: 4463
Людей с доходом от 100 до 200 тыс. рублей: 11924
Людей с доходом более 200 тыс. рублей: 5067


Как видим, распределение похоже на нормальное. Выделим на основании этого деления четыре типа доходов - низкие, средне-низкие, средне-высокие и высокие - и запишем тип дохода для каждой строки в отдельный столбец.

In [24]:
#Пишем функцию, возвращающую тип дохода
def income_category(income):
    try:
        if 0<income<100000:
            return 'низкий'
        elif 100000<=income<=150000:
            return 'средне-низкий'
        elif 150000<income<=200000:
            return 'средне-высокий'
        if income>200000:
            return 'высокий'
    except:
        return 'ошибка'

#Применяем функцию к столбцу total_income и записываем результат в новый столбец total_income_id
data['total_income_type']=data['total_income'].apply(income_category)
#Убеждаемся в том, что функция сработала правильно
print(data.head(10))


   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   среднее             1   
8         2   -6929.865299         35    высшее             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

 Перейдем к целям получения кредита. Дла начала посмотрим, какие варианты и с какой частотой встречаются среди выделенных нами лемм.

In [25]:
from collections import Counter
print(Counter(data['purpose_lemmas'].sum()))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Попробуем рассортировать значимые для нас леммы по категориям. Например, "жилье" и "недвижимость" отнесем в одну категорию, если только вместе со словом "недвижимость" не соседствует слово "коммерческий" - предполагаем, что по умолчанию под недвижимостью люди имеют в виду именно жилую.

In [26]:
#Пишем функцию, выделяющую категории
def lemma_category(category):
        if 'жилье' in category:
            return 'жилье'
        elif ('коммерческий' in category) & ('недвижимость' in category):
            return 'коммерческая недвижимость'
        elif ('недвижимость' in category) & ('коммерческий' not in category):
            return 'жилье'
        elif 'свадьба'in category:
            return 'свадьба'
        elif 'ремонт' in category:
            return 'ремонт'
        elif 'образование' in category:
            return 'образование'
        elif 'автомобиль' in category:
            return 'автомобиль'

    
#Применим функцию к нашим данным, результаты запишем в новый столбец purpose_type
data['purpose_type']=data['purpose_lemmas'].apply(lemma_category)
#Подсчитаем, сколько записей каждой категории было создано:
print(data['purpose_type'].value_counts())
#Посмотрим, остались ли записи, не вошедшие ни в одну категорию:
print('Всего записей: ',data['purpose_type'].value_counts().sum())

        

жилье                        9500
автомобиль                   4306
образование                  4013
свадьба                      2324
коммерческая недвижимость    1311
Name: purpose_type, dtype: int64
Всего записей:  21454


Как видим, нам повезло и записей, не вошедших ни в одну из этих категорий, нет. 

В данных есть две пары столбцов, которые фактически дублируют друг друга: это education и education_id, а также family_status и fimily_status_id. Мы можем выделить текстовые значения в отдельные словари, а в общей таблице оставить только численные. Это позволит уменьшить размер и улучшить читаемость основного набора данных.

In [27]:
#Начнем с образования
education_dictionary=data[['education','education_id']]
#Удалим в словаре дубликаты, оставим только уникальные значения
education_dictionary=education_dictionary.drop_duplicates().reset_index(drop=True)
#Посмотрим, что получилось
print(education_dictionary)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [28]:
#Все верно. Переходим к семейному положению
family_status_dictionary=data[['family_status','family_status_id']]
#Удаляем дубликаты
family_status_dictionary=family_status_dictionary.drop_duplicates().reset_index(drop=True)
#Смотрим, что получилось
print(family_status_dictionary)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


In [29]:
#Последний этап - сохраняем набор данных уже без лишних столбцов
data=data[['children','days_employed','dob_years','education_id','family_status_id','gender','income_type','debt','total_income','total_income_type','purpose','purpose_lemmas','purpose_type']]
print(data.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1   -8437.673028         42             0                 0      F   
1         1   -4024.803754         36             1                 0      F   
2         0   -5623.422610         33             1                 0      M   
3         3   -4124.747207         32             1                 0      M   
4         0  340266.072047         53             1                 1      F   
5         0    -926.185831         27             0                 1      M   
6         0   -2879.202052         43             0                 0      F   
7         0    -152.779569         50             1                 0      M   
8         2   -6929.865299         35             0                 1      F   
9         0   -2188.756445         41             1                 0      M   

  income_type  debt  total_income total_income_type  \
0   сотрудник     0        253875           высокий   
1   сотру

### Вывод

Мы категоризировали данные в интересующих нас столбцах, касающихся цели получения кредита и уровня дохода, что позволит нам удобно работать с этими данными в будущем. Также дублирующиеся столбцы были убраны во внешние словари. Кроме того, мы пришли к выводу о том, как категоризировать ошибочгые данные в столбце количества детей.

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Разделим записи на две группы - "есть дети" и "нет детей" - и посчитаем процент имеющих задолженность для каждой из них. Начнем с тех, у кого дети есть.

In [30]:
#Как мы выяснили выше, записи с ошибочным количеством детей логично отнести к группе "есть дети"/
data_have_children=data[data['children']!=0]['debt']
#Делим количество имеющих задолженность людей с детьми на общее количество людей с детьми
have_children_have_debt=data_have_children.sum()/len(data_have_children)
#Посмотрим, что получилось
print('Задолженность есть у {:.1%} людей с детьми'.format(have_children_have_debt))

Задолженность есть у 9.2% людей с детьми


Теперь все то же самое для людей, у которых детей нет.

In [31]:
data_no_children=data[data['children']==0]['debt']
no_children_have_debt=data_no_children.sum()/len(data_no_children)
print('Задолженность есть у {:.1%} людей без детей'.format(no_children_have_debt))

Задолженность есть у 7.5% людей без детей


#### Вывод

Люди, у которых нет детей, с большей вероятностью вернут кредит в срок. Вероятно, наличие детей заставляет семьи брать кредиты без трезвой оценки своих финансовых возможностей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводную таблицу для разных типов семейного положения и соответствующего им процента наличия задолженности.

In [32]:
#Построим первичную сводную таблицу из имеющихся данных
family_status_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='gender', aggfunc='count')
#Добавим колонку percentage - процент имеющих задолженность по каждой из категорий
family_status_pivot['percentage']=family_status_pivot[1]/family_status_pivot[0]*100

#Облагородим внешний вид таблицы - добавим внешний словарь и оставим только нужные столбцы
family_status_pivot=family_status_pivot.merge(family_status_dictionary, on='family_status_id', how='left')
family_status_pivot_final=family_status_pivot[['family_status','percentage']]
family_status_pivot_final.set_axis(['Семейное положение','Процент имеющих задолженность'],axis = 'columns',inplace = True)

#Посмотрим, что получилось
print(family_status_pivot_final.sort_values(by='Процент имеющих задолженность', ascending=False))


      Семейное положение  Процент имеющих задолженность
4  Не женат / не замужем                      10.804416
1       гражданский брак                      10.310922
0        женат / замужем                       8.160940
3              в разводе                       7.657658
2         вдовец / вдова                       7.031250


In [37]:
#Построим первичную сводную таблицу из имеющихся данных
family_status_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='gender', aggfunc='count')
#Добавим колонку percentage - процент имеющих задолженность по каждой из категорий
family_status_pivot['percentage']=family_status_pivot[1]/family_status_pivot[0]
family_status_pivot['percentage'] = family_status_pivot['percentage'].map('{:.2%}'.format)
#Облагородим внешний вид таблицы - добавим внешний словарь и оставим только нужные столбцы
family_status_pivot=family_status_pivot.merge(family_status_dictionary, on='family_status_id', how='left')
family_status_pivot_final=family_status_pivot[['family_status','percentage']]
family_status_pivot_final.set_axis(['Семейное положение','Процент имеющих задолженность'],axis = 'columns',inplace = True)

#Посмотрим, что получилось
family_status_pivot_final.sort_values(by='Процент имеющих задолженность', ascending=False)

,Семейное положение,Процент имеющих задолженность
0,женат / замужем,8.16%
3,в разводе,7.66%
2,вдовец / вдова,7.03%
4,Не женат / не замужем,10.80%
1,гражданский брак,10.31%


#### Вывод

Как видим, наибольший процент имеющих задолженность по кредитам - у неженатых и незамужних, а наименьший - у вдов и вдовцов. Возможно, это связано с тем, что среднестатистическому неженатому незачем брать кредит, а в нашу выборку попали в основном люди, находящиеся в сложной ситуации или начинающие бизнес - это подтверждается небольшим количеством неженатых в нашей выборке в целом.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Повторим алгоритм из предыдущего шага - построим сводную таблицу для разных типов семейного положения и соответствующего им процента наличия задолженности.

In [33]:
#Построим первичную сводную таблицу из имеющихся данных
total_income_type_pivot = data.pivot_table(index=['total_income_type'], columns='debt', values='gender', aggfunc='count')
#Добавим колонку percentage - процент имеющих задолженность по каждой из категорий
total_income_type_pivot['percentage']=total_income_type_pivot[1]/total_income_type_pivot[0]*100
#Поработаем над внешним видом таблицы
total_income_type_pivot.reset_index(inplace=True)
total_income_type_pivot_final=total_income_type_pivot[['total_income_type','percentage']]
total_income_type_pivot_final.set_axis(['Уровень дохода','Процент имеющих задолженность'],axis = 'columns',inplace = True)
#Посмотрим на получившийся итог
print(total_income_type_pivot_final.sort_values(by='Процент имеющих задолженность', ascending=False))


   Уровень дохода  Процент имеющих задолженность
3   средне-низкий                       9.547124
2  средне-высокий                       9.291122
1          низкий                       8.615235
0         высокий                       7.602463


#### Вывод

Что не удивительно, самый низкий процент имеющих задолженность по кредитам - среди людей с высоким доходом (более 200 тыс.). Но что удивительно - у людей с низким доходом (до 100 тыс. рублей) он ниже, чем у людей как со средне-низким (100-150 тыс.), так и со средне-высоким (150-200 тыс.) доходом. Самый высокий процент имеющих задолженности - у людей со средне-низкими доходами. Возможно, это связано с тем, что по мере роста дохода запросы и финансовые ожидания людей растут быстрее, чем их реальное благосостояние.

- Как разные цели кредита влияют на его возврат в срок?

Используем примерно тот же алгоритм, что и на двух предыдущих шагах - построим сводную таблицу для разных целей взятия кредита и соответствующего им процента наличия задолженности.

In [34]:
#Построим первичную сводную таблицу из имеющихся данных
purpose_type_pivot = data.pivot_table(index=['purpose_type'], columns='debt', values='gender', aggfunc='count')
#Добавим колонку percentage - процент имеющих задолженность по каждой из категорий
purpose_type_pivot['percentage']=purpose_type_pivot[1]/purpose_type_pivot[0]*100
#Поработаем над внешним видом таблицы
purpose_type_pivot.reset_index(inplace=True)
purpose_type_pivot_final=purpose_type_pivot[['purpose_type','percentage']]
purpose_type_pivot_final.set_axis(['Цель кредита','Процент имеющих задолженность'],axis = 'columns',inplace = True)
#Посмотрим на получившийся итог
print(purpose_type_pivot_final.sort_values(by='Процент имеющих задолженность', ascending=False))

                Цель кредита  Процент имеющих задолженность
0                 автомобиль                      10.325391
3                образование                      10.156464
4                    свадьба                       8.699719
2  коммерческая недвижимость                       8.168317
1                      жилье                       7.746399


#### Вывод

Наибольший процент должников - среди взявших автокредит. Видимо, люди делают это для того, чтобы "пустить пыль в глаза", и не оценивают реалистично свои финансовые возможности. Следом по количеству должников к сожалению идет кредит на образование. Но возможно, что дело не невозможности найти хорошую работу параллельно с обучением. Самый низкий процент должников - среди берущих кредит на жилую недвижимость. Видимо, квартира в залоге хорошо прививает финансовую дисциплину.

### Шаг 4. Общий вывод

Выше мы показали, что все факторы, вопросы о которых были заданы, влияют на вероятность погашения кредита в срок. 
Люди без детей становятся должниками с меньшей вероятностью, чем люди с детьми. Неженатые и незамужние чаще не выплачивают кредит вовремя, чем находящиеся в браке, разведенные или вдовы/вдовцы. Уровень дохода влияет на платежеспособность нелинейно: самые надежные заемщики - люди с высоким доходом, но далее по степени надежности следуют люди с низкими доходами, а уже после них - со средними. Наконец, автокредит и кредит на образование будут выплачены в срок с наименьшей вероятностью, а ипотечный кредит - с наибольшей.
Также хотелось бы обратить внимание на то, что работа с данными в столбце "трудовой стаж" на стороне банка или их выгрузка организованы неправильно. Если бы перед нами стояла задача оценить влияние этого столбца на платежеспособность, мы бы скорее всего не смогли этого сделать.